# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df_lng_lat = pd.read_csv("../output_data/lng_lat.csv")
df_lng_lat.head()
df_lng_lat['humidity'].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = df_lng_lat[['lat','lon']]
weights = df_lng_lat["humidity"]

In [21]:
# Plot Heatmap
fig = gmaps.figure(center=(31.6315 ,-8.0083), zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100, point_radius=1)


# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Setting my ideal vacation parameters 
vaca_weather = df_lng_lat.loc[(df_lng_lat['max_temp']<= 75) & (df_lng_lat['max_temp']>= 45)]
vaca_weather = df_lng_lat.loc[(df_lng_lat['humidity']<= 20)]
vaca_weather.reset_index(inplace=True, drop = True)
vaca_weather.drop('Unnamed: 0', axis='columns', inplace=True)
len(vaca_weather)

/Users/jds/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
df_hotel = pd.DataFrame(vaca_weather)
df_hotel.head()

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605
1,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682
2,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760
3,Ati,18.3353,13.2154,97.07,18,68,4.00,TD,1620843925
4,Qena,32.7183,26.1615,87.12,11,0,10.07,EG,1620844003


In [32]:
#Adding new columns to DF
df_hotel['hotel_name'] = ""
df_hotel['hotel_address'] = ""
df_hotel['hotel_rating'] = ""
df_hotel.head()

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,,,
1,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,,,
2,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760,,,
3,Ati,18.3353,13.2154,97.07,18,68,4.00,TD,1620843925,,,
4,Qena,32.7183,26.1615,87.12,11,0,10.07,EG,1620844003,,,


In [33]:
#for debugging
# df_hotel = df_hotel[0:10]
# df_hotel

In [34]:
params = {
    "types": 'lodging',
    "radius": 5000,
    "key": g_key
}

for index, row in df_hotel.iterrows():
    lat = row['lat']
    lng = row['lon']
    
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    #print(name_address.url)
    try:
        df_hotel.loc[index, "hotel_name"] = name_address["results"][0]["name"]
        df_hotel.loc[index, "hotel_address"] = name_address["results"][0]["vicinity"]
        df_hotel.loc[index, "hotel_rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [35]:
df_hotel

,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,Expeditions Ténére voyages,Bilma,4.8
1,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,,,
2,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760,Hotel Benjamín Hill,"Sin nombre No. 12 LB, INFONAVIT, Benjamín Hill",
3,Ati,18.3353,13.2154,97.07,18,68,4.00,TD,1620843925,,,
4,Qena,32.7183,26.1615,87.12,11,0,10.07,EG,1620844003,"Grand hotel ""Quena""","Al Gomhoureya, Qism Qena, Qena",3.7
5,Saint George,-113.5841,37.1041,71.40,12,1,1.01,US,1620843881,Best Western Plus Abbey Inn,"1129 South Bluff Street, St. George",4.6
6,Adrar,-10.0711,20.5022,99.86,8,0,15.86,MR,1620844078,,,
7,Arlit,7.3853,18.7369,101.08,5,10,6.58,NE,1620844158,Case de Passage De la Prefecture,"RN25, Arlit",4.5
8,Iranshahr,60.6848,27.2025,88.36,12,0,13.80,IR,1620844226,پلکه بزرگ,ايرانشهر، ايرانشهر، استان سیستان و بلوچستان، ا...,
9,Atar,-13.0499,20.5169,99.48,9,5,15.23,MR,1620844231,Oumou Elghoura,"Mbarka Oumara Derriere l'hopital regional, Atar",3.4


In [36]:
#Save fig to csv, but then comment out the save because it will override the csv if you run the notebook over again.
#df_hotel.to_csv("vacation_hotel.csv")
df_hotel = pd.read_csv('vacation_hotel.csv')

In [37]:
df_hotel

,Unnamed: 0,city,lon,lat,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,hotel_address,hotel_rating
0,0,Bilma,12.9164,18.6853,100.51,7,22,6.85,NE,1620843605,Expeditions Ténére voyages,Bilma,4.8
1,1,Tessalit,1.0114,20.1986,104.27,6,36,11.88,ML,1620843682,NaN,NaN,NaN
2,2,Benjamín Hill,-111.1000,30.1667,82.47,9,0,8.88,MX,1620843760,Hotel Benjamín Hill,"Sin nombre No. 12 LB, INFONAVIT, Benjamín Hill",NaN
3,3,Ati,18.3353,13.2154,97.07,18,68,4.00,TD,1620843925,NaN,NaN,NaN
4,4,Qena,32.7183,26.1615,87.12,11,0,10.07,EG,1620844003,"Grand hotel ""Quena""","Al Gomhoureya, Qism Qena, Qena",3.7
5,5,Saint George,-113.5841,37.1041,71.40,12,1,1.01,US,1620843881,Best Western Plus Abbey Inn,"1129 South Bluff Street, St. George",4.6
6,6,Adrar,-10.0711,20.5022,99.86,8,0,15.86,MR,1620844078,NaN,NaN,NaN
7,7,Arlit,7.3853,18.7369,101.08,5,10,6.58,NE,1620844158,Case de Passage De la Prefecture,"RN25, Arlit",4.5
8,8,Iranshahr,60.6848,27.2025,88.36,12,0,13.80,IR,1620844226,پلکه بزرگ,ايرانشهر، ايرانشهر، استان سیستان و بلوچستان، ا...,NaN
9,9,Atar,-13.0499,20.5169,99.48,9,5,15.23,MR,1620844231,Oumou Elghoura,"Mbarka Oumara Derriere l'hopital regional, Atar",3.4


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotel.iterrows()]
locations1 = df_hotel[["lat", "lon"]]

In [41]:
# Plot Heatmap
marker_layer = gmaps.symbol_layer(locations=locations1, fill_color='blue', info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))